In [10]:
# This is code takes input as your interested STR regions and SNP loci and check if any read in the a perticular STR region is enclosing the read and the same time the Snp loci (matching the snp allele with the reference or alternative allele) finally it gives the variation occured in STR region which is associating with the Snp allele.

# packages

import gzip
from tqdm import tqdm
from collections import Counter
import pandas as pd
import os
import numpy as np
import itertools
import re
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean
import random
from collections import OrderedDict
import pysam
import sys


# Below block is the main function which calls the other functions like (snp_allele_origin_classification(), snp_check(), STR_variation_from_read(), Snp_AssociatedSTR_GT())

# Dictionary which contains Trio id's and it's relavent Off-spring id's
# list_of_files = '../../Analysis_AV/1KG_Trio_Samples.tsv'
# files_information = {}

# with open(f'{list_of_files}') as file_info:
#     file_info.readline()

#     for Trio_id in file_info:
#         Trio_id = Trio_id.strip().split('\t')
#         files_information[Trio_id[0]]= Trio_id[2]


        
# Calling the argument using os.sys[]
# Trio_id = sys.argv[1]


# Read and Write directory.
Read_directory = '../../data_pysam/cram/'
Write_directory = '../../data_pysam/cram/'
Pysam_Read_directory = '../../data_pysam/cram/'

def Trio_file_id(Trio_id):
    
    single_file_id = Trio_id
    # pysam_infile = files_information[Trio_id1]
    with open(f'{Read_directory}{single_file_id}_HM_intermediate.tsv') as Trio_file:

        out = open(f'{Write_directory}{single_file_id}_HM_Phased_mDNMs.tsv', 'w')

        samfile = pysam.AlignmentFile(f'{Pysam_Read_directory}{single_file_id}.final.cram', "rc")

        for repeat_region in tqdm(Trio_file):

            repeat_region = repeat_region.strip().split('\t')
            #
            start =  int(repeat_region[1])  
            end =    int(repeat_region[2])
            chrom = repeat_region[0]

            snp_pos = int(repeat_region[-5]) - 1
            snp_gts = [repeat_region[-4], repeat_region[-3]]
            snp_gts_representation = [repeat_region[-8], repeat_region[-7], repeat_region[-6]]
            denovo_gts = repeat_region[5].split('|') # change the var name : denovo_gts to offspring alleles
            repeat_len = end - start

            # here we are calling the function to store the snp allele parent of origin.
            snp_allele_origin = snp_allele_origin_classification(snp_gts, snp_gts_representation)

            # dictionary to maintian the snp_loc and repeat region enclosing the reads.
            read_info = {}

            #
            Read_depth_5percent = 0    
            for read_depth,read in enumerate(samfile.fetch(chrom,start, end)):

                # read quality >= 1.
                if read.mapping_quality >= 1: #******* changed the read mapping quality.
                    Read_depth_5percent += 1

                    Read = ''
                    if read.is_read1: Read = 'R1'
                    else: Read = 'R2'

                    read_query = read.query_name
                    read_start = read.reference_start
                    read_end = read.reference_end
                    base_info = sum(read.get_aligned_pairs(with_seq=True), ())

                    # checking if the ( read is enclosed) and ( covering the interested snp position in the flank). { R1:[s,e],[reference pos],[read base at reference pos] }
                    if not read_query in read_info: 
                        read_info[read_query] = {'R1':[], 'R2':[], 'Allele_on':[]}

                        #
                        snp_on_read_info = snp_check(read, snp_pos, Read, snp_gts, base_info)            
                        Read_lenvar_info = STR_variation_from_read(read, start, end, repeat_len)
                        read_info[read_query]['Allele_on'].extend(snp_on_read_info)
                        read_info[read_query][Read].append(Read_lenvar_info)
                        # print(snp_on_read_info)


                    else:
                        snp_on_read_info = snp_check(read, snp_pos, Read, snp_gts, base_info)            
                        Read_lenvar_info = STR_variation_from_read(read, start, end, repeat_len)
                        read_info[read_query]['Allele_on'].extend(snp_on_read_info)
                        read_info[read_query][Read].append(Read_lenvar_info)
                        # print(snp_on_read_info)

            # Calling the function which gives Snps associated with STR alleles.
            Snp_AssociatedSTR_GT_print = Snp_AssociatedSTR_GT(read_info, snp_gts,denovo_gts)

            # printing the required data.
            if len(Snp_AssociatedSTR_GT_print) > 0:
                print(*repeat_region[:5],repeat_region[5],repeat_region[6],repeat_region[7],repeat_region[8],f'{snp_gts[0]}|{snp_gts[1]}',f'{snp_allele_origin[0]}:{snp_allele_origin[1]};{snp_allele_origin[2]}:{snp_allele_origin[3]}', Snp_AssociatedSTR_GT_print,Read_depth_5percent, read_depth+1,repeat_region[-1], repeat_region[-2], file=out, sep = '\t')


        samfile.close()

        out.close()




# **** Function 1: Snp_AssociatedSTR_GT()
# ****************************************************************************************************************************************************************************************
# Function which returns the Denovo genotype assiciating with the Snp genotype bases which further can be related to Denovo allele parent of origin.

def Snp_AssociatedSTR_GT(read_info, snp_gts, denovo_gts):
    
    # A copy of read_info dictionary to delete the read id's which are not usefull. ()
    read_info_copy = read_info.copy()
    Dnv_PofO = []    
    
    # Before entering into the current repeat region read_info dictionary, we must ignore if ( read is not enclosed the repeat region | No snp on any read | or snp on R1 or R2 but no read overlapping the repeat region.
    for read_id in read_info:
        
        if len(read_info[read_id]['Allele_on']) < 2 or len(read_info[read_id]['R1'])==0 and len(read_info[read_id]['R2'])==0 : read_info_copy.pop(read_id)
        elif len(read_info[read_id]['R1']) == 1 and read_info[read_id]['R1'][0] == 0  and  len(read_info[read_id]['R2']) == 1 and read_info[read_id]['R2'][0] == 0 : read_info_copy.pop(read_id)
        elif read_info[read_id]['Allele_on'].count(0) > 1: read_info_copy.pop(read_id)
    
    # Snp_alleles_to_DnvAllele_counts dict gets updated with SNP alleles and STR lens coming from those reads.
    Snp_alleles_to_DnvAllele_counts = {snp_gts[0]:[], snp_gts[1]:[]} 
    
    # Updating the Snp_alleles_to_DnvAllele_counts dict with STR allele lens.
    list_DnvAlleles = [Snp_alleles_to_DnvAllele_counts[read_info_copy[read_ID]['Allele_on'][1].upper()].extend(read_info_copy[read_ID]['R1'] + read_info_copy[read_ID]['R2']) for read_ID in read_info_copy.keys() if read_info_copy[read_ID]['Allele_on'][0] != 0]
    list_DnvAlleles1 = [Snp_alleles_to_DnvAllele_counts[read_info_copy[read_ID]['Allele_on'][2].upper()].extend(read_info_copy[read_ID]['R1'] + read_info_copy[read_ID]['R2']) for read_ID in read_info_copy.keys() if read_info_copy[read_ID]['Allele_on'][0] == 0]
    
 
    # In here we are considering the major count allele associating with Snp allele.
    for i,snp_allele in enumerate(Snp_alleles_to_DnvAllele_counts.keys()):
        
        # In here we are doing Counter
        snp_allele_itr = Counter(Snp_alleles_to_DnvAllele_counts[snp_allele])
        if len(snp_allele_itr) == 0 : continue
        
        #
        if int(denovo_gts[0]) in snp_allele_itr or int(denovo_gts[1]) in snp_allele_itr:
            
            # Deleting the 0's from the list of alleles, where 0 means that perticular read is not enclosing the repeat region.
            if 0 in snp_allele_itr: snp_allele_itr.pop(0) 
            
            #
            if len(list(snp_allele_itr)) == 0: continue
            
            values = list(snp_allele_itr.values())
            keys = list(snp_allele_itr.keys())

            #
            index_max_value = values.index(max(values))            
            Dnv_AlleleAssociated_snp = keys[index_max_value]
            
            #
            if Dnv_AlleleAssociated_snp != 0:
                Dnv_allele_depth = max(values)
                Dnv_PofO.append([Dnv_AlleleAssociated_snp,snp_allele,Dnv_allele_depth])# Here we want to print not just the max read supporting allele, we print list of allele which are getting printed.
        
        
    
    return Dnv_PofO



# Function2 : snp_check()
# ****************************************************************************************************************************************************************************************
# Creating function to check the snp_pos overlapping with which read and matching with which snp_allele

def snp_check(read, snp_pos, Read, snp_gts, base_info):
    
    # base info = [read_pos, refn_pos, read_base]
    reference_pos = list(base_info)[1::3]
    read_bases = list(base_info)[2::3]
    
    # check if read is overalpped with snp_pos and base.
    if snp_pos in reference_pos: # with the only confirmation that read.get_aligned_pairs() gives information like 'read_pos, reference_pos, base' for matched, deletion and substitution bases for other bases it reports None.
        read_index = list(base_info).index(snp_pos)
        # Here we are ignoring the deletion in snp_pos, coz as we are only considering substitutions from the snp_vcf file.
        if base_info[read_index-1] != None: 
            read_base = list(str(read.query_sequence))[base_info[read_index-1]]

            # checking if snp_pos base matchs with read base in same pos.
            if read_base.upper() in snp_gts:
                 return [Read, read_base]
        
            else: return []
        
        else: return []
        
    else: return []




# Function 3 : STR_variation_from_read()
# ****************************************************************************************************************************************************************************************
# Considering only Repeat overlapped CIGAR variants.

def STR_variation_from_read(read, start, end, repeat_len):

    
    # 1st we check if the read is enclosing the repeat or not, if not then return the empty list.
    if read.reference_start <= int(start)  and  read.reference_end >= int(end):
        
        # Finding out the way to get the CIGAR string pos in read, reference & type of variation.
        # Then figuring out the way to include the way to fecth the info in our set of reads.

        repeat_start = int(start)
        repeat_end = int(end)


        cigar_s = read.cigarstring
        cigar_t = sum(read.cigartuples, ())

        num_bases = list(cigar_t)[1::2]
        alignment_type = list(cigar_t)[::2]

        # We update while itterating through the read positions to cover the entire read positions with any len and capture the variations which encounter within the repeat region.
        current_ref_pos = read.reference_start

        # saving Insertions and deletions counts which are part of repeat region in the list.
        change_in_len_insert = {'1':0}
        change_in_len_del = []

        for indx in range(len(alignment_type)): # this loop runs for every read, itterating through M's I's D's.

            # Here we get the actual reference pos by substracting the reference pos with hard or soft clip bases.
            if alignment_type[indx] == 1:
                # adding the insertion counts to the dictionary change_in_len
                if current_ref_pos in range(repeat_start-1, repeat_end+1) :
                    change_in_len_insert['1'] += num_bases[indx]


            elif alignment_type[indx] == 2:
                # adding the deletion counts to the dict.
                del_end_pos = (current_ref_pos + num_bases[indx])
                l = [(change_in_len_del.append(1)) for del_pos in range(current_ref_pos+1, del_end_pos+1) if del_pos in range(repeat_start, repeat_end+1)] 
                current_ref_pos += num_bases[indx]


            elif alignment_type[indx] not in [4,5]:
                current_ref_pos += num_bases[indx]


        # change in the len of read overlapping the repeat.
        Insertion_substract_Deletion = int(change_in_len_insert['1']) - sum(change_in_len_del)
        len_diff_in_read = repeat_len + Insertion_substract_Deletion


        return len_diff_in_read


    else: return 0



# Function 4 : snp_allele_origin_classification()
# ****************************************************************************************************************************************************************************************
# Here we relate snp alleles to the maternal and paternal side.

def snp_allele_origin_classification(snp_gts, snp_gts_representation):
    
    Snp_origin = []
    
    # We have to 0 or 1 being unique in list of GT combinations.
    offspring = snp_gts_representation[0].split('|')
    parents = snp_gts_representation[1].split('|') + snp_gts_representation[2].split('|')
    GT_bases = {'0':snp_gts[0], '1':snp_gts[1]}
    
    # Based on the '0' and '1' counts, either being count 1 where that allele indicates it is coming from criteria satisfied condition.
    p1 =  parents.count(offspring[0])
    p2 = parents.count(offspring[1])
    
    #
    paternal = snp_gts_representation[1]#.split('|')
    maternal = snp_gts_representation[2]#.split('|')

    # To findout the 0 or 1 coming from which parent.
    if p1 == 1 and paternal.count(offspring[0]) == 1:
        Snp_origin.extend([GT_bases[offspring[0]], 'P', GT_bases[offspring[1]], 'M'])
    
    elif p1 == 1 and maternal.count(offspring[0]) == 1:
        Snp_origin.extend([GT_bases[offspring[0]], 'M', GT_bases[offspring[1]], 'P'])

    elif paternal.count(offspring[1]) == 1:
        Snp_origin.extend([GT_bases[offspring[1]], 'P', GT_bases[offspring[0]], 'M'])

    else:  Snp_origin.extend([GT_bases[offspring[1]], 'M', GT_bases[offspring[0]], 'P'])


    return Snp_origin



# Passsing the Trio file name as an argument to the main function.
Trio_file_id('HG03540')

488it [00:12, 38.13it/s]


In [22]:
for file in tqdm(os.listdir('../../Analysis_KT/SNP_Denovo/SNPs_Intermediate_Files/SNPs_AssociatedRepeats/')):
    
    if file.endswith('_Test.tsv.gz'):
        Trio_file_id(file)

  0%|                                                                                                                                                                                  | 0/5 [00:01<?, ?it/s]


ValueError: could not determine alignment length